# Home Credit Analysis

Goal of this notebook is to answer:

- What factors will affect how much an individual is approved for?
- How does a previous application affect your loan amount?
- Which loan purpose, increase or decrease the amount difference?
- What start time of application, increase or decrease the amount difference? 

Data: https://www.kaggle.com/c/home-credit-default-risk/data

# Importing Data

In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
from matplotlib import pyplot as pyplot

from google.colab import drive
drive.mount('/content/drive')
drive_dir = '/content/drive/Shared drives/Project 4 (MATH 3439)/Data/'

Mounted at /content/drive


#Loading the Data

## Train / Test

Not using but good to compare.

In [ ]:
#df1_train = pd.read_csv(drive_dir + 'application_train.csv.zip')
#df2_test = pd.read_csv(drive_dir + 'application_test.csv.zip')

## Loading data from each data frame (6 total).

### installments_payments

In [ ]:
sample_df_instalpay = pd.read_csv(drive_dir + 'installments_payments.csv.zip', nrows=100000).sample(80000)
# Load 1000 rows 200 times
for i in range(10):
    new_sample_df_instalpay = pd.read_csv(drive_dir + 'installments_payments.csv.zip', nrows=100000, skiprows=range(1,(i+1)*100000)).sample(80000)

    sample_df_instalpay = pd.concat([sample_df_instalpay, new_sample_df_instalpay])

In [ ]:
#Select duplicate rows except first occurrence based on all columns
duplicateRowsDF = sample_df_instalpay[sample_df_instalpay.duplicated()]
print("Duplicate Rows except first occurrence based on all columns are :")
print(duplicateRowsDF)

Duplicate Rows except first occurrence based on all columns are :
   SK_ID_PREV  SK_ID_CURR  ...  AMT_INSTALMENT  AMT_PAYMENT
0     2149338      111259  ...        28730.25     28730.25

[1 rows x 8 columns]


In [ ]:
sample_df_instalpay.head()

SK_ID_PREV  SK_ID_CURR  ...  AMT_INSTALMENT  AMT_PAYMENT
85438     2599877      174840  ...       10716.210    10716.210
87430     1102197      138433  ...        2250.000     2250.000
16596     2012087      121332  ...       25584.525    25584.525
86535     2792257      127624  ...        3758.670     3758.670
81800     1560899      168735  ...       32495.985    32495.985

[5 rows x 8 columns]

In [ ]:
sample_df_instalpay['SK_ID_PREV'].nunique()

236558

In [ ]:
sample_df_instalpay.shape

(880000, 8)

In [ ]:
sample_df_instalpay.to_csv(drive_dir + 'installments_payments_mini.csv', index = False )

### POS_CASH_balance

In [ ]:
sample_df_poscashb = pd.read_csv(drive_dir + 'POS_CASH_balance.csv.zip', nrows=100000).sample(80000)
# Load 1000 rows 200 times
for i in range(10):
    new_sample_df_poscashb = pd.read_csv(drive_dir + 'POS_CASH_balance.csv.zip', nrows=100000, skiprows=range(1,(i+1)*100000)).sample(80000)

    sample_df_poscashb = pd.concat([sample_df_poscashb, new_sample_df_poscashb])

In [ ]:
#Select duplicate rows except first occurrence based on all columns
duplicateRowsDF = sample_df_poscashb[sample_df_poscashb.duplicated()]
print("Duplicate Rows except first occurrence based on all columns are :")
print(duplicateRowsDF)

Duplicate Rows except first occurrence based on all columns are :
   SK_ID_PREV  SK_ID_CURR  ...  SK_DPD  SK_DPD_DEF
0     1671450      186011  ...       0           0

[1 rows x 8 columns]


In [ ]:
sample_df_poscashb.head()

SK_ID_PREV  SK_ID_CURR  ...  SK_DPD  SK_DPD_DEF
19765     1239342      216180  ...       0           0
76957     2179568      397416  ...       0           0
67581     2501225      257924  ...       0           0
82658     2023169      170080  ...       0           0
84403     1374439      148961  ...       0           0

[5 rows x 8 columns]

In [ ]:
sample_df_poscashb['SK_ID_PREV'].nunique()

477172

In [ ]:
sample_df_poscashb.shape

(880000, 8)

In [ ]:
sample_df_poscashb.to_csv(drive_dir + 'POS_CASH_balance_mini.csv', index = False )

### bureau

In [ ]:
sample_df_bureau = pd.read_csv(drive_dir + 'bureau.csv.zip', nrows=100000).sample(80000)
# Load 1000 rows 200 times
for i in range(10):
    new_sample_df_bureau = pd.read_csv(drive_dir + 'bureau.csv.zip', nrows=100000, skiprows=range(1,(i+1)*100000)).sample(80000)

    sample_df_bureau = pd.concat([sample_df_bureau, new_sample_df_bureau])

In [ ]:
#Select duplicate rows except first occurrence based on all columns
duplicateRowsDF = sample_df_bureau[sample_df_bureau.duplicated()]
print("Duplicate Rows except first occurrence based on all columns are :")
print(duplicateRowsDF)

Duplicate Rows except first occurrence based on all columns are :
   SK_ID_CURR  SK_ID_BUREAU  ... DAYS_CREDIT_UPDATE AMT_ANNUITY
0      240757       6542162  ...                 -6     10125.0

[1 rows x 17 columns]


In [ ]:
sample_df_bureau.head()

SK_ID_CURR  SK_ID_BUREAU  ... DAYS_CREDIT_UPDATE AMT_ANNUITY
22634      406090       5197457  ...               -414       0.000
39025      171262       5811086  ...               -233         NaN
2308       167780       5717336  ...                -15    6120.000
42492      249764       5212718  ...                -18   13500.000
75796      164484       5253659  ...               -271    8657.325

[5 rows x 17 columns]

In [ ]:
sample_df_bureau.shape

(880000, 17)

In [ ]:
sample_df_bureau.to_csv(drive_dir + 'bureau.csv_mini.csv', index = False )

### bureau_balance

In [ ]:
sample_df_bureaubal = pd.read_csv(drive_dir + 'bureau_balance.csv.zip', nrows=100000).sample(80000)
# Load 1000 rows 200 times
for i in range(10):
    new_sample_df_bureaubal = pd.read_csv(drive_dir + 'bureau_balance.csv.zip', nrows=100000, skiprows=range(1,(i+1)*100000)).sample(80000)

    sample_df_bureaubal = pd.concat([sample_df_bureaubal, new_sample_df_bureaubal])

In [ ]:
#Select duplicate rows except first occurrence based on all columns
duplicateRowsDF = sample_df_bureaubal[sample_df_bureaubal.duplicated()]
print("Duplicate Rows except first occurrence based on all columns are :")
print(duplicateRowsDF)

Duplicate Rows except first occurrence based on all columns are :
   SK_ID_BUREAU  MONTHS_BALANCE STATUS
0       5241763             -27      X


In [ ]:
sample_df_bureaubal.head()

SK_ID_BUREAU  MONTHS_BALANCE STATUS
44147       5262488             -28      X
55297       6021653             -87      0
49897       6020603             -44      C
74326       5726348              -5      0
68578       5004181             -23      0

In [ ]:
sample_df_bureaubal['SK_ID_BUREAU'].nunique()

32432

In [ ]:
sample_df_bureaubal.shape

(880000, 3)

In [ ]:
sample_df_bureaubal.to_csv(drive_dir + 'bureau_balance_mini.csv', index = False )

### credit_card_balance

In [ ]:
sample_df_creditcb = pd.read_csv(drive_dir + 'credit_card_balance.csv.zip', nrows=100000).sample(80000)
# Load 1000 rows 200 times
for i in range(10):
    new_sample_df_creditcb = pd.read_csv(drive_dir + 'credit_card_balance.csv.zip', nrows=100000, skiprows=range(1,(i+1)*100000)).sample(80000)

    sample_df_creditcb = pd.concat([sample_df_creditcb, new_sample_df_creditcb])

In [ ]:
sample_df_creditcb.head()

SK_ID_PREV  SK_ID_CURR  ...  SK_DPD  SK_DPD_DEF
94319     1730221      401839  ...       0           0
67327     2657934      366032  ...       0           0
33661     2411238      257442  ...       0           0
25298     1698374      353507  ...       0           0
39766     1214346      322324  ...       0           0

[5 rows x 23 columns]

In [ ]:
#Select duplicate rows except first occurrence based on all columns
duplicateRowsDF = sample_df_creditcb[sample_df_creditcb.duplicated()]
print("Duplicate Rows except first occurrence based on all columns are :")
print(duplicateRowsDF)

Duplicate Rows except first occurrence based on all columns are :
   SK_ID_PREV  SK_ID_CURR  ...  SK_DPD  SK_DPD_DEF
0     2710147      381634  ...       0           0

[1 rows x 23 columns]


In [ ]:
sample_df_creditcb['SK_ID_PREV'].nunique()

98320

In [ ]:
df_credprev_nunique = sample_df_creditcb[sample_df_creditcb['SK_ID_PREV']==2657934]

In [ ]:
df_credprev_nunique.head()

SK_ID_PREV  SK_ID_CURR  ...  SK_DPD  SK_DPD_DEF
67327     2657934      366032  ...       0           0
14838     2657934      366032  ...       0           0
2297      2657934      366032  ...       1           1
59009     2657934      366032  ...       1           1
21033     2657934      366032  ...       0           0

[5 rows x 23 columns]

In [ ]:
sample_df_creditcb['SK_ID_CURR'].nunique()

97692

In [ ]:
sample_df_creditcb.shape

(880000, 23)

In [ ]:
sample_df_creditcb.to_csv(drive_dir + 'credit_card_balance_mini.csv', index = False )

### previous_application

In [ ]:
sample_df_app = pd.read_csv(drive_dir + 'previous_application.csv.zip', nrows=100000).sample(80000)
# Load 10000 rows 200 times
for i in range(10):
    new_sample_df_app = pd.read_csv(drive_dir + 'previous_application.csv.zip', nrows=100000, skiprows=range(1,(i+1)*100000)).sample(80000)

    sample_df_app = pd.concat([sample_df_app, new_sample_df_app])

In [ ]:
sample_df_app.head()

SK_ID_PREV  SK_ID_CURR  ... DAYS_TERMINATION  NFLAG_INSURED_ON_APPROVAL
80082     2138331      249458  ...            -50.0                        0.0
42593     1989262      446664  ...          -1555.0                        0.0
54142     2007043      170000  ...           -656.0                        0.0
39009     2392160      335249  ...              NaN                        NaN
45060     2808823      193643  ...              NaN                        NaN

[5 rows x 37 columns]

In [ ]:
#Select duplicate rows except first occurrence based on all columns
duplicateRowsDF = sample_df_app[sample_df_app.duplicated()]
print("Duplicate Rows except first occurrence based on all columns are :")
print(duplicateRowsDF)

Duplicate Rows except first occurrence based on all columns are :
   SK_ID_PREV  SK_ID_CURR  ... DAYS_TERMINATION  NFLAG_INSURED_ON_APPROVAL
0     1792520      267933  ...           -356.0                        1.0

[1 rows x 37 columns]


In [ ]:
sample_df_app['SK_ID_PREV'].nunique()

879999

In [ ]:
sample_df_app.shape

(880000, 37)

In [ ]:
df_app_nunique = sample_df_app[sample_df_app['SK_ID_CURR']==388788]

In [ ]:
df_app_nunique.head()

SK_ID_PREV  SK_ID_CURR  ... DAYS_TERMINATION  NFLAG_INSURED_ON_APPROVAL
70959     1238555      388788  ...          -2332.0                        0.0
88546     1845466      388788  ...          -1618.0                        0.0

[2 rows x 37 columns]

In [ ]:
sample_df_app['NAME_CONTRACT_STATUS'].unique()

array(['Approved', 'Canceled', 'Refused', 'Unused offer'], dtype=object)

In [ ]:
sample_df_app.to_csv(drive_dir + 'previous_application_mini.csv', index = False )

# Merging

In [ ]:
result = pd.merge(sample_df_app, sample_df_poscashb, how='left', on=['SK_ID_PREV'])

In [ ]:
result.head()

SK_ID_PREV  SK_ID_CURR_x  ... SK_DPD  SK_DPD_DEF
0     2138331        249458  ...    NaN         NaN
1     1989262        446664  ...    NaN         NaN
2     2007043        170000  ...    NaN         NaN
3     2392160        335249  ...    NaN         NaN
4     2808823        193643  ...    NaN         NaN

[5 rows x 44 columns]

In [ ]:
result = result.dropna(how='any')

In [ ]:
result.shape

(4, 44)

In [ ]:
result.head()

SK_ID_PREV  SK_ID_CURR_x  ... SK_DPD  SK_DPD_DEF
213566      1468627        394419  ...    0.0         0.0
1016540     1897803        259229  ...    0.0         0.0
1016541     1897803        259229  ...    0.0         0.0
1071324     2677591        159789  ...    0.0         0.0

[4 rows x 44 columns]

# Cleaning the Data

In [ ]:
sample_df_app.head()

SK_ID_PREV  SK_ID_CURR  ... DAYS_TERMINATION  NFLAG_INSURED_ON_APPROVAL
44971     1923284      317363  ...           -138.0                        1.0
763       1923756      198658  ...            -35.0                        0.0
23379     2084613      201578  ...              NaN                        NaN
93314     1843857      227309  ...         365243.0                        0.0
15754     1930776      214211  ...           -195.0                        1.0

[5 rows x 37 columns]

There are no missing values, as shown below:

In [ ]:
for c in sample_df_instalpay.columns:
  num_missing = sample_df_instalpay[c].isna().sum()
  if num_missing > 0:
    print(f'{c}: {num_missing} ({100*num_missing / sample_df_instalpay.shape[0]:.2f}%) missing values')

In [ ]:
sample_df_poscashb.head()

SK_ID_PREV  SK_ID_CURR  ...  SK_DPD  SK_DPD_DEF
99822     1292450      416728  ...       0           0
93824     2755361      339018  ...       0           0
47831     2748646      312916  ...       0           0
96530     1481716      326683  ...       0           0
77297     2525737      409921  ...       0           0

[5 rows x 8 columns]

In [ ]:
for c in sample_df_poscashb.columns:
  num_missing = sample_df_poscashb[c].isna().sum()
  if num_missing > 0:
    print(f'{c}: {num_missing} ({100*num_missing / sample_df_poscashb.shape[0]:.2f}%) missing values')

CNT_INSTALMENT: 403 (0.18%) missing values
CNT_INSTALMENT_FUTURE: 402 (0.18%) missing values


- `CNT_INSTALMENT`:Filled missing values with 0 because no credit history
- `CNT_INSTALMENT_FUTURE`: Filled missing values with 0 because no previous credit

In [ ]:
sample_df_poscashb = sample_df_poscashb.fillna(0)

In [ ]:
sample_df_bureau.head()

SK_ID_CURR  SK_ID_BUREAU  ... DAYS_CREDIT_UPDATE AMT_ANNUITY
71290      419493       5248091  ...               -941         NaN
96267      206103       6537713  ...               -283         NaN
55983      214218       5233626  ...               -685         NaN
83669      133011       5728008  ...                 -7         NaN
41358      204862       5211336  ...                -57         NaN

[5 rows x 17 columns]

Checking for missing values

In [ ]:
for c in sample_df_bureau.columns:
  num_missing = sample_df_bureau[c].isna().sum()
  if num_missing > 0:
    print(f'{c}: {num_missing} ({100*num_missing / sample_df_bureau.shape[0]:.2f}%) missing values')

DAYS_CREDIT_ENDDATE: 13672 (6.21%) missing values
DAYS_ENDDATE_FACT: 81909 (37.23%) missing values
AMT_CREDIT_MAX_OVERDUE: 145387 (66.08%) missing values
AMT_CREDIT_SUM: 1 (0.00%) missing values
AMT_CREDIT_SUM_DEBT: 32803 (14.91%) missing values
AMT_CREDIT_SUM_LIMIT: 76921 (34.96%) missing values
AMT_ANNUITY: 152724 (69.42%) missing values


- DAYS_CREDIT_ENDDATE: Remaining duration of CB credit (in days) at the time of application in Home Credit 
- DAYS_ENDDATE_FACT: Days since CB credit ended at the time of application in Home Credit (only for closed credit) 
- AMT_CREDIT_MAX_OVERDUE: Maximal amount overdue on the Credit Bureau credit so far (at application date of loan in our sample) 
- AMT_CREDIT_SUM_DEBT: Current debt on Credit Bureau credit 
- AMT_CREDIT_SUM_LIMIT: Current credit limit of credit card reported in Credit Bureau 
- AMT_ANNUITY: Annuity of the Credit Bureau credit

In [ ]:
sample_df_bureau = sample_df_bureau.fillna(0)

In [ ]:
sample_df_bureaubal.head()

SK_ID_BUREAU  MONTHS_BALANCE STATUS
39134       5258568             -95      X
74328       5726348              -7      0
47448       6020189             -29      C
92417       5235796             -10      C
30346       5230761               0      0

In [ ]:
for c in sample_df_bureaubal.columns:
  num_missing = sample_df_bureaubal[c].isna().sum()
  if num_missing > 0:
    print(f'{c}: {num_missing} ({100*num_missing / sample_df_bureaubal.shape[0]:.2f}%) missing values')

In [ ]:
sample_df_creditcb.head()

SK_ID_PREV  SK_ID_CURR  ...  SK_DPD  SK_DPD_DEF
90214     1239151      196844  ...       0           0
34375     1308598      237350  ...       1           1
56217     1460002      209577  ...       0           0
18431     2164166      427657  ...       0           0
12663     2417748      291479  ...       0           0

[5 rows x 23 columns]

In [ ]:
for c in sample_df_creditcb.columns:
  num_missing = sample_df_creditcb[c].isna().sum()
  if num_missing > 0:
    print(f'{c}: {num_missing} ({100*num_missing / sample_df_creditcb.shape[0]:.2f}%) missing values')

AMT_DRAWINGS_ATM_CURRENT: 50132 (22.79%) missing values
AMT_DRAWINGS_OTHER_CURRENT: 50132 (22.79%) missing values
AMT_DRAWINGS_POS_CURRENT: 50132 (22.79%) missing values
AMT_INST_MIN_REGULARITY: 13467 (6.12%) missing values
AMT_PAYMENT_CURRENT: 50762 (23.07%) missing values
CNT_DRAWINGS_ATM_CURRENT: 50132 (22.79%) missing values
CNT_DRAWINGS_OTHER_CURRENT: 50132 (22.79%) missing values
CNT_DRAWINGS_POS_CURRENT: 50132 (22.79%) missing values
CNT_INSTALMENT_MATURE_CUM: 13467 (6.12%) missing values


Never had credit?

In [ ]:
sample_df_creditcb = sample_df_creditcb.fillna('unknown')

In [ ]:
sample_df_app.head()

SK_ID_PREV  SK_ID_CURR  ... DAYS_TERMINATION  NFLAG_INSURED_ON_APPROVAL
44971     1923284      317363  ...           -138.0                        1.0
763       1923756      198658  ...            -35.0                        0.0
23379     2084613      201578  ...              NaN                        NaN
93314     1843857      227309  ...         365243.0                        0.0
15754     1930776      214211  ...           -195.0                        1.0

[5 rows x 37 columns]

In [ ]:
for c in sample_df_app.columns:
  num_missing = sample_df_app[c].isna().sum()
  if num_missing > 0:
    print(f'{c}: {num_missing} ({100*num_missing / sample_df_app.shape[0]:.2f}%) missing values')

AMT_ANNUITY: 46850 (21.30%) missing values
AMT_DOWN_PAYMENT: 112482 (51.13%) missing values
AMT_GOODS_PRICE: 48174 (21.90%) missing values
RATE_DOWN_PAYMENT: 112482 (51.13%) missing values
RATE_INTEREST_PRIMARY: 219221 (99.65%) missing values
RATE_INTEREST_PRIVILEGED: 219221 (99.65%) missing values
NAME_TYPE_SUITE: 107027 (48.65%) missing values
CNT_PAYMENT: 46850 (21.30%) missing values
PRODUCT_COMBINATION: 35 (0.02%) missing values
DAYS_FIRST_DRAWING: 84769 (38.53%) missing values
DAYS_FIRST_DUE: 84769 (38.53%) missing values
DAYS_LAST_DUE_1ST_VERSION: 84769 (38.53%) missing values
DAYS_LAST_DUE: 84769 (38.53%) missing values
DAYS_TERMINATION: 84769 (38.53%) missing values
NFLAG_INSURED_ON_APPROVAL: 84769 (38.53%) missing values
